In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Normal Prior, Normal Likelihood - Mean and Covariance Unknown

The Normal($\theta$) with $ \theta = (\mu, \sigma^2)$ is reparametrized according to the exponential family's natural parameters:

$ \eta(\theta) = \eta(\mu, \sigma^2) = (\mu / \sigma^2, -1 / 2\sigma^2) = (\eta_1, \eta_2) $

$ \eta^{-1}(\eta_1, \eta_2) = (-\eta_1 / 2\eta_2, -1 / 2\eta_2)$

Therefore, the prior is on $\eta$ and of the form

$ \pi (\eta; \alpha, R) \propto \exp \left(-\frac{1}{2}(\eta - \alpha)^T R^{-1}(\eta - \alpha)   \right) $,

where we need to specify $\alpha$ and $R$, the prior mean and prior covariance (mean on $\eta$, not $\theta$!).

In [8]:
#useful functions

def eta(mu, sigma2):
    """computes eta(theta) = eta(mu, sigma2)"""
    return np.array([[mu/sigma2, -1/(2*sigma2)]]).T

def eta_jac(mu, sigma2):
    return np.array([ [1/sigma2, -mu/(sigma2**2)],
                      [0.         , 1/(2*sigma2**2)] ]
                    )

def eta_inv(eta1, eta2):
    """recovers mu, sigma^2 (**NOT** sigma only, its square)"""
    return np.array([-eta1/(2*eta2), -1/(2*eta2) ])

def eta_inv_jac(eta1, eta2):
    """compute the jacobian of eta^{-1}(eta1, eta2)"""
    return np.array([ [-1/(2*eta2), eta1/(2*eta2**2)],
                        [0.         , 1/(2*eta2**2)] ]
                    )

def log_normal_density(x, mu, sigma2):
    return 1 / np.sqrt(x**2 * sigma2 * 2 * np.pi) * np.exp( (-1/2) * (np.log(x) - mu)**2 / sigma2 )

def normal_density(x, mu, sigma2):
    return 1 / np.sqrt(sigma2 * 2 * np.pi) * np.exp( (-1/2) * (x - mu)**2 / sigma2 )

def MV_normal(x, mu, Sigma):
    return np.exp((-1/2) * (x - mu).T @ np.linalg.inv(Sigma) @ (x - mu))

In [3]:
class Normal():
    def __init__(self, eta_prior_mean, eta_prior_cov):
        self.eta_prior_mean = eta_prior_mean #prior on eta
        self.eta_prior_cov = eta_prior_cov #prior on cov of eta
        self.num_params = eta_prior_mean.size

        self.dr_ = lambda x : np.array([[1., 2 * x]])
        self.db_ = lambda x : 0.
        self.ddr_ = lambda x : np.array([[0., 2.]])
        
        return
    
    def posterior(self, data):
        assert data.ndim == 1, "Data must be 1-dimensional"

        Lambda = np.zeros((self.num_params, self.num_params), dtype=float)
        Nu = np.zeros(self.eta_prior_mean.shape)

        T = len(data)

        for x in data:
            Lambda += (1/T) * self.dr_(x).T @ self.dr_(x)
            Nu += (2/T) * (self.dr_(x).T * self.db_(x) + self.ddr_(x).T)

        eta_post_cov = np.linalg.inv(np.linalg.inv(self.eta_prior_cov) + 2 * T * Lambda)
        eta_post_mean = eta_post_cov @ (np.linalg.inv(self.eta_prior_cov) @ self.eta_prior_mean - T * Nu)

        return eta_post_mean.flatten(), eta_post_cov

In [12]:
#fix a random seed for experimental purposes
np.random.seed(0)

#the "true parameters", and the generated data
mu = 1.0
sigma2 = 0.7**2

size = 100
data = np.random.normal(loc=mu, scale=np.sqrt(sigma2), size=size)

#Setting the prior on theta = (mu, sigma2)
mu_prior, sigma2_prior = 0.0, 1.0
alpha, beta = 2, 1
theta_prior_mean = np.array([mu_prior, sigma2_prior])
theta_prior_cov = np.array([[alpha, 0], [0, beta]])

#converting the prior mu and sigma2 to natural parameters

eta_prior_mean = eta(*theta_prior_mean)
eta_prior_cov = eta_jac(mu_prior, sigma2_prior) @ theta_prior_cov @ eta_jac(mu_prior, sigma2_prior).T

#instantiating the LogNormal class and obtaining the posterior mean and covariance for the r.v. \eta | data 
norm = Normal(eta_prior_mean=eta_prior_mean, eta_prior_cov=eta_prior_cov)
eta_post_mean, eta_post_cov = norm.posterior(data=data)

#converting back from eta to mu, sigma2, and obtaining the mean and covariance for mu and sigma2
theta_post_mean = eta_inv(*eta_post_mean)
theta_post_cov = eta_inv_jac(*eta_post_mean) @ eta_post_cov @ eta_inv_jac(*eta_post_mean).T

NN_sigma2 = data.var() #Normal-Normal likelihood covariance (known)
NN_theta_post = (1 / (1/sigma2_prior + size/NN_sigma2) ) * (mu_prior/sigma2_prior + data.sum()/NN_sigma2)


theta_post_mean_formula = data.mean() / (1 + sigma2_prior**2 / (2 * size * alpha))

print("naive frequentist estimate (mean, var)             : ", [data.mean(), data.var()])
print("theta from Normal-Normal (known var) (mean, var)   : ", np.array([NN_theta_post, NN_sigma2]))
print()
print("theta from Normal-Normal (unknown var) (mean, var) : ", theta_post_mean)
print("formula                                            : ", theta_post_mean_formula)
print("theta (true)                                       : ", np.array([mu, sigma2]))

naive frequentist estimate (mean, var)             :  [1.0418656108741395, 0.49775504341531646]
theta from Normal-Normal (known var) (mean, var)   :  [1.03670536 0.49775504]

theta from Normal-Normal (unknown var) (mean, var) :  [1.03926744 0.50294725]
formula                                            :  1.0392674422684685
theta (true)                                       :  [1.   0.49]
